In [2]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [3]:
# from music21 import converter,instrument # or import *
# file = converter.parse('midi_data/elise.mid')
# components = []
# for element in file.recurse():
#     components.append(element)
#     print(components)
#     print()

In [4]:
def get_notes():

    notes = []

    for file in glob.glob("midi_data/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    
    return notes

In [5]:
#import necessary submodules from music21
# from music21 import converter, instrument, note, chord

# notes = []

# for file in glob.glob("./midi_songs/*.mid"):
#     #read the midi file into a music21 stream object
#     midi = converter.parse(file)
#     notes_to_parse = None

#     parts = instrument.partitionByInstrument(midi)
# #     print(len(parts.parts))
#     #if there are any parts
#     if parts:
#         notes_to_parse = parts.parts[0].recurse()
#     else:
#         notes_to_parse = midi.flat.notes

#     for element in notes_to_parse:
#         if isinstance(element,note.Note):     #if it is a note
#             notes.append(str(element.pitch))
#         elif isinstance(element,chord.Chord):   #if it is a chord
#             notes.append('.'.join(str(n) for n in element.normalOrder))

In [7]:
# print(notes)

In [8]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [9]:
# seq_length = 100

# # get all pitch names
# pitchnames = sorted(set(item for item in notes))

#  # create a dictionary to map pitches to integers
# note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

# nw_input = []
# nw_output = []

# # create input sequences and the corresponding outputs
# for i in range(0, len(notes) - seq_length, 1):
#     seq_in = notes[i:i + seq_length]
#     seq_out = notes[i + seq_length]
#     nw_input.append([note_to_int[char] for char in seq_in])
#     nw_output.append(note_to_int[seq_out])

# n_patterns = len(nw_input)

# print(nw_input)
# # reshape the input into a format compatible with LSTM layers
# nw_input = np.reshape(nw_input, (n_patterns, seq_length, 1))
# # normalize input
# nw_input = nw_input / float(n_vocab)
# print("#################################################3")
# print(nw_input)
# nw_output = np_utils.to_categorical(nw_output)

In [11]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [12]:
# model = Sequential()
# model.add(LSTM(512,input_shape=(nw_input.shape[1],nw_input.shape[2]),return_sequences = True))
# model.add(Dropout(0.3))
# model.add(LSTM(512,return_sequences=True))
# model.add(Dropout(0.3))
# model.add(LSTM(256))
# model.add(Dense(256))
# model.add(Dropout(0.3))
# model.add(Dense(n_vocab))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy',optimizer='rmsprop')
# model.load_weights('weights.hdf5')
# model.summary()

In [13]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=3, batch_size=64, callbacks=callbacks_list)

In [14]:
#now creating checkpoints to store the parameters from the latest epoch

# filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

# checkpoint = ModelCheckpoint(filepath,monitor='loss',verbose=0,save_best_only=True,mode='min')

# callbacks_list = [checkpoint]

# model.fit(nw_input,nw_output,epochs=2,batch_size=64,callbacks=callbacks_list)

In [15]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)


In [16]:
train_network()

Parsing midi_data/appass_3.mid
Parsing midi_data/pathetique_2.mid
Parsing midi_data/beethoven_opus22_1.mid
Parsing midi_data/appass_1.mid
Parsing midi_data/waldstein_1.mid
Parsing midi_data/beethoven_opus90_1.mid
Parsing midi_data/beethoven_hammerklavier_2_format0.mid
Parsing midi_data/waldstein_3.mid
Parsing midi_data/beethoven_les_adieux_1.mid
Parsing midi_data/waldstein_2.mid
Parsing midi_data/beethoven_hammerklavier_4.mid
Parsing midi_data/elise.mid
Parsing midi_data/beethoven_hammerklavier_3.mid
Parsing midi_data/beethoven_opus90_2.mid
Parsing midi_data/appass_2.mid
Parsing midi_data/mond_2.mid
Parsing midi_data/mond_3.mid
Parsing midi_data/beethoven_opus22_2.mid
Parsing midi_data/pathetique_3.mid
Parsing midi_data/beethoven_hammerklavier_1.mid
Parsing midi_data/beethoven_opus10_1.mid
Parsing midi_data/beethoven_opus22_4.mid
Parsing midi_data/pathetique_1.mid
Parsing midi_data/beethoven_les_adieux_3.mid
Parsing midi_data/mond_1.mid
Parsing midi_data/beethoven_les_adieux_2.mid
Pars